In [2]:
from bs4 import BeautifulSoup 
import requests
import numpy as np
import pandas as pd
import re

In [7]:


url = "https://en.wikipedia.org/wiki/List_of_official_languages_by_country_and_territory"

text = requests.get(url).text

        
        


In [14]:
countries = "Austria, Belgium, Bulgaria, Croatia, Cyprus, Czechia, Czech Republic, Denmark, Estonia, Finland, France, Germany, Greece, Hungary, Ireland, Italy, Latvia, Lithuania, Luxembourg, Malta, Netherlands, Poland, Portugal, Romania, Slovakia, Slovenia, Spain, Sweden"
eu_countries = countries.split(", ")

In [4]:
def parse_language(repl):
    return re.sub("\(([^\)]+)\)", "", repl.strip().split("[")[0].replace("\n", ", ")).strip()

In [8]:
soup = BeautifulSoup(text)
raw_data = soup.find_all("table")[0].find_all("tbody")[0].find_all("tr")[1:]

languages = pd.DataFrame()
for row in raw_data:
    data = row.find_all("td")
    country = parse_language(data[0].text)
    official_lang = parse_language(data[1].text).split(", ")
    minority_lang = parse_language(data[3].text).split("," )
    widely_spoken_lang = ""

    if (len(data) >= 6):
        widely_spoken_lang = parse_language(data[5].text).split(", ")
    
    print(f'Country: {country}')
    
    

    if country in eu_countries:

        row = {"country" : country, "official_lang" : official_lang, "minority_lang" : minority_lang, "widely_spoken_lang": widely_spoken_lang}
        row_series = pd.Series(row)
        languages = pd.concat([languages, row_series.to_frame().T])
        #print(f'Country: {country}')

languages


Country: Abkhazia
Country: Afghanistan
Country: Albania
Country: Algeria
Country: Andorra
Country: Angola
Country: Antigua and Barbuda
Country: Argentina
Country: Armenia
Country: Australia
Country: Austria
Country: Azerbaijan
Country: Bahamas
Country: Bahrain
Country: Bangladesh
Country: Barbados
Country: Belarus
Country: Belgium
Country: Belize
Country: Benin
Country: Bhutan
Country: Bolivia
Country: Bosnia and Herzegovina
Country: Botswana
Country: Brazil
Country: Brunei
Country: Bulgaria
Country: Burkina Faso
Country: Burundi
Country: Cambodia
Country: Cameroon
Country: Canada
Country: Cape Verde
Country: Central African Republic
Country: Chad
Country: Chile
Country: China
Country: Christmas Island
Country: Cocos  Islands
Country: Colombia
Country: Comoros
Country: Democratic Republic of the Congo
Country: Republic of the Congo
Country: Cook Islands
Country: Costa Rica
Country: Croatia
Country: Cuba
Country: Cyprus
Country: Czech Republic
Country: Denmark
Country: Djibouti
Country:

,country,official_lang,minority_lang,widely_spoken_lang
0,Austria,[German],"[Slovene, Czech, Hungarian, Slovak, Romani...",[English]
0,Belgium,"[Dutch, French, German]",[],[English]
0,Bulgaria,[Bulgarian],[],[]
0,Croatia,[Croatian],[],[English]
0,Cyprus,[Greek],[],[English]
0,Czech Republic,[Czech],[Belarusian],[]
0,Denmark,[Danish],[German],[English]
0,Estonia,[Estonian],[],[Russian]
0,Finland,"[Finnish, Swedish]",[Sami],[English]
0,France,[French],"[Corsican language, Basque language, Breton ...",[Occitan language]


In [30]:
url = "https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population_density"

html = requests.get(url).text

pop_dens_soup = BeautifulSoup(html)

rows = pop_dens_soup.find_all("tbody")[0]

rows = rows.find_all("tr")[1:]

countries = []
pop_density = []


for row in rows:
    data = row.find_all("td")
    country = data[0].text.strip()
    pop_dens_km = float(data[1].text.strip().replace(",", ""))

    #special case, need specifically metropolitan france
    if country ==  "France (metropolitan)":
        country = "France"
    
    if country in eu_countries:
        countries.append(country)
        pop_density.append(pop_dens_km)

pd.DataFrame({"country" : countries, "pop_density" : pop_density})

    



,country,pop_density
0,Malta,1700.0
1,Netherlands,424.0
2,Belgium,383.0
3,Luxembourg,253.0
4,Germany,233.0
5,Italy,195.0
6,Denmark,138.0
7,Cyprus,136.0
8,Czech Republic,133.0
9,Poland,120.0


Units are per km^2

In [6]:
url  = "https://worldpopulationreview.com/country-rankings/best-healthcare-in-the-world"

html = requests.get(url).text

healthcare_soup = BeautifulSoup(html)


<table class="tp-table-body is-narrow w-full min-w-full table-auto border-separate border-spacing-0 border bg-white"><thead class="relative z-40 bg-sky-100 px-4 py-3 font-bold"><tr><th class="datatable-th align-bottom sticky top-0 border-b border-gray-400 bg-sky-50 bg-clip-padding px-2 py-1 pin left-0 z-50 border-l-0 text-left"><div class="flex flex-col justify-end items-stretch"><div class="flex items-center cursor-pointer select-none">Country</div><div><div class="flex flex-col"><input class="w-36 border shadow rounded-sm px-1 py-0.5 text-sm" placeholder="Search... (195)" type="text" value=""/><div class="h-1"></div></div></div></div></th><th class="datatable-th align-bottom sticky top-0 border-b border-gray-400 bg-sky-50 bg-clip-padding px-2 py-1 z-40"><div class="flex flex-col justify-end items-stretch"><div class="flex items-center cursor-pointer select-none">Legatum Prosperity Index Health Score<svg class="icon icon-tabler icon-tabler-chevron-down" fill="none" height="18" stroke=

In [16]:
countries = []
scores = []

for tr in healthcare_soup.find_all("table")[1].find_all("tr")[1:]:
    country = tr.find("th").text
    score = tr.find_all("td")[1].text
    if country in eu_countries:
        countries.append(country)
        scores.append(score)

pd.DataFrame({"country" : countries, "healthcare_index" : scores})

,country,healthcare_index
0,Sweden,56.29
1,Netherlands,54.63
2,Luxembourg,56.31
3,Germany,55.98
4,Finland,60.53
5,Denmark,55.46
6,Italy,72.15
7,Belgium,53.99
8,Malta,53.59
9,France,55.31


In [37]:
url = "https://www.numbeo.com/cost-of-living/rankings_by_country.jsp?title=2024"

html = requests.get(url).text

col_soup = BeautifulSoup(html)

In [38]:
countries = []
scores = []

for tr in col_soup.find_all("table")[1].find_all("tr")[1:]:
    data = tr.find_all("td")
    
    
    country = data[1].text
    col = data[2].text
    if country in eu_countries:
        countries.append(country)
        scores.append(col)

pd.DataFrame({"country" : countries, "COL_index" : scores})

,country,COL_index
0,Denmark,75.2
1,Austria,66.8
2,Ireland,66.5
3,France,66.0
4,Finland,65.5
5,Luxembourg,65.3
6,Netherlands,64.3
7,Belgium,63.0
8,Germany,62.7
9,Sweden,62.2


In [39]:
url = "https://en.wikipedia.org/wiki/List_of_countries_by_average_yearly_temperature"

html = requests.get(url).text

temp_soup = BeautifulSoup(html)

In [61]:
for tr in temp_soup.find_all("tbody")[0].find_all("tr")[1:]:
    data = tr.find_all("td")
    country = data[1].text
    temp = re.sub(r'[^0-9.]', "", data[2].text.split(" ")[0])
    if country in eu_countries:
        countries.append(country)
        temps


29.26
29.21
29.17
28.90
28.82
28.71
28.62
28.49
28.40
28.38
28.17
28.11
28.04
28.02
28.02
28.01
27.98
27.97
27.95
27.90
27.83
27.82
27.81
27.77
27.71
27.71
27.71
27.69
27.68
27.66
27.64
27.63
27.61
27.60
27.58
27.47
27.47
27.41
27.38
27.33
27.30
27.30
27.28
27.25
27.20
27.09
27.00
26.98
26.95
26.95
26.85
26.83
26.80
26.79
26.70
26.63
26.61
26.58
26.55
26.54
26.49
26.38
26.31
26.29
26.27
26.17
26.12
26.06
25.96
25.94
25.92
25.91
25.88
25.86
25.81
25.75
25.71
25.71
25.70
25.60
25.58
25.54
25.47
25.45
25.44
25.23
25.20
25.08
25.04
25.03
25.01
25.00
24.97
24.95
24.94
24.83
24.80
24.79
24.74
24.74
24.72
24.71
24.68
24.66
24.57
24.55
24.49
24.44
24.41
24.35
24.30
24.16
23.92
23.82
23.73
23.65
23.60
23.37
23.33
23.25
23.14
22.95
22.92
22.81
22.69
22.66
22.64
22.53
22.23
22.09
22.05
21.90
21.77
21.67
21.43
21.38
21.31
20.76
20.64
20.56
20.53
20.51
20.45
19.99
19.72
19.58
19.52
19.50
19.20
19.01
18.95
18.75
18.34
18.23
18.15
18.14
18.10
17.97
16.66
16.34
15.85
15.45
15.11
14.26
14.17
14.07
13.6